In [1]:
import praw
from dotenv import dotenv_values
import re
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import pandas as pd
from datetime import datetime

In [9]:

config = dotenv_values(".env")

reddit = praw.Reddit(
        client_id='K04HQ-znvcgyFH5iyrVKVw',
        client_secret='rjIM9tjt1rBwAUaPD9gDEfoup63buQ',
        user_agent='Metric Analysis'
    )
subredit_name = 'stocks'
posts = reddit.subreddit(subredit_name).hot(limit=10)

title = {subredit_name + '_' + str(i+1): post.title for i, post in enumerate(posts)}
    
print(title)

Version 7.7.1 of praw is outdated. Version 7.8.1 was released Friday October 25, 2024.


{'stocks_1': 'r/Stocks Daily Discussion & Fundamentals Friday Nov 15, 2024', 'stocks_2': 'r/Stocks Daily Discussion & Fundamentals Friday Nov 29, 2024', 'stocks_3': '23-Hour Party People: U.S. stocks may soon trade nearly around the clock.', 'stocks_4': 'Thoughts on buying BRB.K this month instead of DCAing into S&P?', 'stocks_5': 'Automate stop loss ', 'stocks_6': 'Before Santa / December Rally Begins', 'stocks_7': 'Technology and its effect on stock market average returns. Does the future hold higher average returns?', 'stocks_8': 'Fellow Canadians: What’s the Difference Between ZSP and VFV?', 'stocks_9': 'Reddit overtakes X in popularity of social media platforms in UK $RDDT', 'stocks_10': 'Is there a realistic scenario where you would sell all your stocks? '}


In [ ]:
import nltk

# Download the VADER lexicon
nltk.download('vader_lexicon')

# Initialize VADER Sentiment Analyzer
sid = SentimentIntensityAnalyzer()

posts_to_analyze = 100  # Number of posts to extract
data = []

# Regular expression for stock ticker detection (assuming uppercase 1-5 letters)
ticker_regex = r'\b[A-Z]{1,5}\b'


def extract_data():
    for post in reddit.subreddit(subredit_name).hot(limit=posts_to_analyze):
        post.comments.replace_more(limit=0)  # Flatten comment tree
        comments = [comment.body for comment in post.comments.list()]
        ticker_matches = re.findall(ticker_regex, post.title)
        
        # Sentiment Analysis
        title_sentiment = sid.polarity_scores(post.title)
        post_data = {
            "Title": post.title,
            "Author": post.author.name if post.author else None,
            "Upvotes": post.score,
            "Date": datetime.utcfromtimestamp(post.created_utc).strftime('%Y-%m-%d %H:%M:%S'),
            "Sentiment_Positive": title_sentiment['pos'],
            "Sentiment_Negative": title_sentiment['neg'],
            "Sentiment_Neutral": title_sentiment['neu'],
            "Sentiment_Compound": title_sentiment['compound'],
            "Ticker_Mentions": list(set(ticker_matches)),  # Unique tickers in the title
            "Comment_Count": len(comments),
            "Comments": comments  # Add comments to the data
        }
        data.append(post_data)

extract_data()

# Save to a DataFrame
df = pd.DataFrame(data)

# Show the DataFrame
print(df.head())

# Save to CSV
df.to_csv("reddit_stocks_analysis.csv", index=False)


[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /home/hadoop/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
/tmp/ipykernel_125307/2219212216.py:27: DeprecationWarning: datetime.datetime.utcfromtimestamp() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.fromtimestamp(timestamp, datetime.UTC).
  "Date": datetime.utcfromtimestamp(post.created_utc).strftime('%Y-%m-%d %H:%M:%S'),


                                               Title             Author  \
0  r/Stocks Daily Discussion & Fundamentals Frida...      AutoModerator   
1  r/Stocks Daily Discussion & Fundamentals Frida...      AutoModerator   
2  23-Hour Party People: U.S. stocks may soon tra...     DecisionNo1902   
3  Thoughts on buying BRB.K this month instead of...  Life-Ambition1432   
4  Technology and its effect on stock market aver...       DawgCheck421   

   Upvotes                 Date  Sentiment_Positive  Sentiment_Negative  \
0       21  2024-11-15 10:30:09               0.000                 0.0   
1        7  2024-11-29 10:30:09               0.000                 0.0   
2      370  2024-11-29 18:01:21               0.197                 0.0   
3       47  2024-11-29 14:54:01               0.000                 0.0   
4       15  2024-11-29 16:35:52               0.000                 0.0   

   Sentiment_Neutral  Sentiment_Compound Ticker_Mentions  Comment_Count  \
0              1.000   

In [16]:
df = pd.read_csv("reddit_stocks_analysis.csv")

In [17]:
df.head()

,Title,Author,Upvotes,Date,Sentiment_Positive,Sentiment_Negative,Sentiment_Neutral,Sentiment_Compound,Ticker_Mentions,Comment_Count,Comments
0,r/Stocks Daily Discussion & Fundamentals Frida...,AutoModerator,21,2024-11-15 10:30:09,0.000,0.0,1.000,0.0000,[],467,"['It’s painful now, but holding onto chip equi..."
1,r/Stocks Daily Discussion & Fundamentals Frida...,AutoModerator,7,2024-11-29 10:30:09,0.000,0.0,1.000,0.0000,[],166,"['ASML is this weird green color today, that c..."
2,23-Hour Party People: U.S. stocks may soon tra...,DecisionNo1902,370,2024-11-29 18:01:21,0.197,0.0,0.803,0.4019,"['S', 'U']",152,['If you’re going to go for 23 hour trading (w...
3,Thoughts on buying BRB.K this month instead of...,Life-Ambition1432,47,2024-11-29 14:54:01,0.000,0.0,1.000,0.0000,"['K', 'P', 'S', 'BRB']",49,"[""Welcome to r/stocks!\n\nFor beginner advice,..."
4,Technology and its effect on stock market aver...,DawgCheck421,15,2024-11-29 16:35:52,0.000,0.0,1.000,0.0000,[],9,['I’m just going to put right here that it’s g...
